In [5]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy as np
import pandas as pd
#from xgboost import XGBClassifier

In [6]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [7]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams_diff(aggregated_data)
# Run script

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline = LogisticRegression()

exported_pipeline.fit(np.array(X), np.array(Y))


In [ ]:
# Create id to team name mappings
teamNames = pd.read_csv('./ncaa-data/Teams.csv')
def teamIDNameMapping():
    team_id_map = {}
    for index, row in teamNames.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    
    return team_id_map

team_id_map = teamIDNameMapping()

In [ ]:
def WinChanceMapping(year):
    
    submission_data = []
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    tourney_teams = tourney_seeds[tourney_seeds['Season'] == year]['Team']

    stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
    
    tourney_teams.sort_values()
    
    for team_1 in tourney_teams:
        for team_2 in tourney_teams:
            if team_1 != team_2:
                prediction = mm_predictor.predict_winner(
                    team_1, team_2, exported_pipeline, 2017, stat_fields)
                label = '2017_' + str(team_1) + '_' + str(team_2)
                submission_data.append([label, prediction[0][0]])
    
    
    teamTeamWinChanceMap = {}
    for line in submission_data:
        title = line[0].split('_')
        team1 = team_id_map[int(title[1])]
        team2 = team_id_map[int(title[2])]
        winRate = str(round(line[1]* 100,2))
        if team1 not in teamTeamWinChanceMap:
            teamTeamWinChanceMap[team1] = {}
        if float(winRate) >= 50:
            if team1 in teamTeamWinChanceMap and team2 in teamTeamWinChanceMap[team1] and teamTeamWinChanceMap[team1][team2] > float(winRate):
                continue
            if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
                # update teamTeammap[team2][team1] to remove that key
                teamTeamWinChanceMap[team2].pop(team1, None)
            teamTeamWinChanceMap[team1][team2] = float(winRate)
            
    return teamTeamWinChanceMap

In [8]:
aggregated_data = pd.concat([season_data, tourney_data])

In [13]:
aggregated_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [34]:
winAd = aggregated_data[['WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGA3', 'WFGM3', 'WFTM', 'WFTA', 'WOR', 
                         'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
loseAd = aggregated_data[['LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGA3', 'LFGM3', 'LFTM', 'LFTA', 'LOR', 
                         'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]

winAd['fgp'] = winAd['WFGM'] / winAd['WFGA'] * 100
winAd['3pp'] = winAd['WFGM3'] / winAd['WFGA3'] * 100
winAd['ftp'] = winAd['WFTM'] / winAd['WFTA'] * 100

for item in ['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA']:
    del winAd[item]

loseAd['fgp'] = loseAd['LFGM'] / loseAd['LFGA'] * 100
loseAd['3pp'] = loseAd['LFGM3'] / loseAd['LFGA3'] * 100
loseAd['ftp'] = loseAd['LFTM'] / loseAd['LFTA'] * 100

for item in ['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA']:
    del loseAd[item]

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [35]:
winAd = winAd.rename(index=str, columns={"WScore": "Score", 'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast',
                                'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF'})

loseAd = loseAd.rename(index=str, columns={"LScore": "Score", 'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast',
                                'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF'})

winAd - loseAd

,3pp,Ast,Blk,DR,LTeamID,OR,PF,Score,Stl,TO,WTeamID,fgp,ftp
0,1.428571,5,-1,2,NaN,4,2,6,-2,5,NaN,5.042290,-11.616162
1,15.000000,9,-2,3,NaN,-5,2,7,-4,1,NaN,6.114588,7.631579
2,32.905983,6,-3,4,NaN,-14,2,12,3,-2,NaN,11.242324,-2.248876
3,6.060606,2,-1,-1,NaN,-11,-5,6,10,-7,NaN,10.633727,1.505376
4,5.357143,0,3,7,NaN,-4,6,6,-3,4,NaN,10.470650,21.652422
5,22.727273,4,0,2,NaN,6,-7,26,5,-10,NaN,2.135774,14.596950
6,-1.666667,3,-3,-3,NaN,4,-3,18,1,-13,NaN,-4.523282,10.622711
7,5.042017,9,-6,10,NaN,5,3,14,-7,4,NaN,11.262985,-2.484472
8,12.121212,-4,0,-4,NaN,-4,5,5,1,-8,NaN,2.122016,11.111111
9,7.983193,1,-3,-1,NaN,5,-5,28,11,-12,NaN,3.530425,20.614035


In [32]:
winAd[winAd.columns[1:]] - loseAd[loseAd.columns[1:]]

,Score,OR,DR,Ast,TO,Stl,Blk,PF,fgp,3pp,ftp
0,6,4,2,5,5,-2,-1,2,5.042290,1.428571,-11.616162
1,7,-5,3,9,1,-4,-2,2,6.114588,15.000000,7.631579
2,12,-14,4,6,-2,3,-3,2,11.242324,32.905983,-2.248876
3,6,-11,-1,2,-7,10,-1,-5,10.633727,6.060606,1.505376
4,6,-4,7,0,4,-3,3,6,10.470650,5.357143,21.652422
5,26,6,2,4,-10,5,0,-7,2.135774,22.727273,14.596950
6,18,4,-3,3,-13,1,-3,-3,-4.523282,-1.666667,10.622711
7,14,5,10,9,4,-7,-6,3,11.262985,5.042017,-2.484472
8,5,-4,-4,-4,-8,1,0,5,2.122016,12.121212,11.111111
9,28,5,-1,1,-12,11,-3,-5,3.530425,7.983193,20.614035


In [50]:
newAd = winAd['WTeamID'].to_frame()
newAd['LTeamID'] = loseAd['LTeamID']
newAd[winAd.columns[1:]] = winAd[winAd.columns[1:]] - loseAd[loseAd.columns[1:]]

In [51]:
newAd

,WTeamID,LTeamID,Score,OR,DR,Ast,TO,Stl,Blk,PF,fgp,3pp,ftp
0,1104,1328,6,4,2,5,5,-2,-1,2,5.042290,1.428571,-11.616162
1,1272,1393,7,-5,3,9,1,-4,-2,2,6.114588,15.000000,7.631579
2,1266,1437,12,-14,4,6,-2,3,-3,2,11.242324,32.905983,-2.248876
3,1296,1457,6,-11,-1,2,-7,10,-1,-5,10.633727,6.060606,1.505376
4,1400,1208,6,-4,7,0,4,-3,3,6,10.470650,5.357143,21.652422
5,1458,1186,26,6,2,4,-10,5,0,-7,2.135774,22.727273,14.596950
6,1161,1236,18,4,-3,3,-13,1,-3,-3,-4.523282,-1.666667,10.622711
7,1186,1457,14,5,10,9,4,-7,-6,3,11.262985,5.042017,-2.484472
8,1194,1156,5,-4,-4,-4,-8,1,0,5,2.122016,12.121212,11.111111
9,1458,1296,28,5,-1,1,-12,11,-3,-5,3.530425,7.983193,20.614035
